In [15]:
import pandas as pd
import json
import numpy

In [12]:
dataset = pd.read_csv("partial_subtracks.csv")

In [13]:
print(dataset[:30])

          lat       lon  id  DBSCAN_labels new_ids
0   41.148522 -8.585676  T1              0      T1
1   41.148639 -8.585712  T1              0      T1
2   41.148855 -8.585685  T1              0      T1
3   41.148927 -8.585730  T1              0      T1
4   41.148963 -8.585982  T1              0      T1
5   41.148954 -8.586396  T1              0      T1
6   41.148720 -8.586072  T1              0      T1
7   41.147847 -8.586324  T1              0      T1
8   41.147460 -8.586999  T1              0      T1
9   41.147154 -8.586576  T1              0      T1
10  41.145570 -8.610876  T2              1      T2
11  41.145579 -8.610858  T2              1      T2
12  41.145768 -8.610903  T2              1      T2
13  41.146191 -8.610444  T2              1      T2
14  41.146758 -8.609445  T2              1      T2
15  41.147118 -8.608896  T2              1      T2
16  41.147127 -8.608968  T2              1      T2
17  41.147532 -8.608707  T2              1      T2
18  41.148117 -8.608347  T2    

In [36]:
def point_distance(lat1, lat2):
    # calculate the distance between the two points
    # distance = 0
    if lat1==lat2:
        return 1
    return -1

In [37]:
gap_penalty = -1

def create_matrix(track1, track2):
    matrix = numpy.zeros((len(track1)+1, len(track2)+1))
    for i in range(0, len(track1)+1):
        matrix[i][0] = gap_penalty * i
    for j in range(0, len(track2)+1):
        matrix[0][j] = gap_penalty * j
    
    for i in range(1, len(track1)+1):
        t1 = track1[i-1]
        for j in range(1, len(track2)+1):
            t2 = track2[j-1]
            match = matrix[i-1][j-1] + point_distance(t1, t2)
            delete = matrix[i-1][j] + gap_penalty
            insert = matrix[i][j-1] + gap_penalty
            matrix[i, j] = max(match, max(delete, insert))
    
    return matrix

matrix =create_matrix('ABCD', 'ADCB')      
print(matrix)

A
B
C
D
[[ 0. -1. -2. -3. -4.]
 [-1.  1.  0. -1. -2.]
 [-2.  0.  0. -1.  0.]
 [-3. -1. -1.  1.  0.]
 [-4. -2.  0.  0.  0.]]


In [45]:
gap_penalty = -1
def backtrack(matrix, track1, track2):
    a1 = []
    a2 = []
    i = len(track1)
    j = len(track2)
    
    while i > 0 or j > 0:
        if i > 0 and j > 0 and matrix[i, j] == matrix[i-1][j-1] + 1:
            a1.insert(0, track1[i])
            a2.insert(0, track2[j])
            i=i-1
            j=j-1
            print('match!', i, j)
        elif i > 0 and matrix[i][j] == matrix[i-1][j] + gap_penalty:
            a1.insert(0, track1[i])
            a2.insert(0, None)
            i=i-1
            print('gap', i, j)
        elif j > 0 and matrix[i][j] == matrix[i][j-1] + gap_penalty:
            a1.insert(0, None)
            a2.insert(0, track2[j])
            j=j- 1
            print('mismatch', i, j)
    return a1, a2

print(backtrack(matrix, 'ABCD', 'ADCB'))

KeyboardInterrupt: 

In [ ]:
def backtrack2(matrix, track1, track2):
    a = []
    start = matrix[track1+1][track2+1]
    

In [ ]:
def align_tracks(track1, track2, gap_penalty):
    """ Needleman-Wunsch algorithm adapted for gps tracks. """
    
    _log.info("Aligning tracks")

    def similarity(p1, p2):
        d = gpxpy.geo.distance(p1.latitude, p1.longitude, p1.elevation,
                               p2.latitude, p2.longitude, p2.elevation)
        return -d

    # construct f-matrix
    f = numpy.zeros((len(track1), len(track2)))
    for i in range(0, len(track1)):
        f[i][0] = gap_penalty * i
    for j in range(0, len(track2)):
        f[0][j] = gap_penalty * j
    for i in range(1, len(track1)):
        t1 = track1[i]
        for j in range(1, len(track2)):
            t2 = track2[j]
            match = f[i-1][j-1] + similarity(t1, t2)
            delete = f[i-1][j] + gap_penalty<
            insert = f[i][j-1] + gap_penalty
            f[i, j] = max(match, max(delete, insert))

    # backtrack to create alignment
    a1 = []
    a2 = []
    i = len(track1) - 1
    j = len(track2) - 1
    while i > 0 or j > 0:
        if i > 0 and j > 0 and \
           f[i, j] == f[i-1][j-1] + similarity(track1[i], track2[j]):
            a1.insert(0, track1[i])
            a2.insert(0, track2[j])
            i -= 1
            j -= 1
        elif i > 0 and f[i][j] == f[i-1][j] + gap_penalty:
            a1.insert(0, track1[i])
            a2.insert(0, None)
            i -= 1
        elif j > 0 and f[i][j] == f[i][j-1] + gap_penalty:
            a1.insert(0, None)
            a2.insert(0, track2[j])
            j -= 1
    return a1, a2

align_tracks()